In [ ]:
!pip install unidic-lite
!pip install --no-binary :all: mecab-python3

     |████████████████████████████████| 47.3MB 104kB/s 
  Created wheel for unidic-lite: filename=unidic_lite-1.0.7-cp36-none-any.whl size=47556593 sha256=e9d96d0a0990ccbd4dc29ef90f754077775dca419be95df1fd68a42f6d02d301
  Stored in directory: /root/.cache/pip/wheels/a8/82/7d/086724645e33a575aafd0b1dae2835c37d2c00c6a0a96ee3a0
Successfully built unidic-lite


In [ ]:
!pip install MeCab-python3

     |████████████████████████████████| 491kB 10.1MB/s 


In [ ]:
!apt install aptitude swig
!aptitude install mecab libmecab-dev mecab-ipadic-utf8 git make curl xz-utils file -y
!git clone --depth 1 https://github.com/neologd/mecab-ipadic-neologd.git
!echo yes | mecab-ipadic-neologd/bin/install-mecab-ipadic-neologd -n -a

In [ ]:
import MeCab
import math
from string import punctuation
from heapq import nlargest
from itertools import product, count
from gensim.models import word2vec
import numpy as np

In [ ]:
from gensim.models.word2vec import Word2Vec

model_path = '/content/word2vec.gensim.model'
model = Word2Vec.load(model_path)
np.seterr(all='warn')

In [ ]:
def cut_sentences(sentence):
  puns = frozenset(u'.!?')
  tmp = []
  for jp in sentence:
    tmp.append(jp)
    if puns.__contains__(jp):
      yield ''.join(tmp)
      tmp = []
  yield ''.join(tmp)

#if need stop words 
## for news i dont think need clear stop words
def create_stopwords():
    stop_list = [line.strip() for line in open("stopwords.txt", 'r', encoding='utf-8').readlines()]
    return stop_list



def two_sentences_similarity(sents_1,sents_2):
  counter = 0
  for sent in sents_1:
    if sent in sents_2:
      counter +=  1
  return counter/(math.log(len(sents_1) + len(sents_2)))

def create_graph(word_sent):
  num = len(word_sent)
  board = [[0.0 for _ in range(num)] for _ in range(num)]
  for i, j in product(range(num), repeat=2):
      if i != j:
          board[i][j] = compute_similarity_by_avg(word_sent[i], word_sent[j])
  return board

def cosine_similarity(vec1,vec2):
  tx = np.array(vec1)
  ty = np.array(vec2)
  cos1 = np.sum(tx * ty)
  cos21 = np.sqrt(tx ** 2)
  cos22 = np.sqrt(ty ** 2)
  f2 = cos21 * cos22
  #print("cos1"+cos1)
  #print("/"+f2)
  cosine_value = cos1 /f2
  return cosine_value

def compute_similarity_by_avg(sents_1,sents_2):
#对两个句子求平均词向量
    if len(sents_1) == 0 or len(sents_2) == 0:
        return 0.0
    if sents_1[0] in model:
      vec1 = model[sents_1[0]]
    else:
      return 0.0
    for word1 in sents_1[1:]:
      if word1 in model:
         vec1 = vec1 + model[word1]
      else:
         vec1 = vec1 + 0.0
      
      if sents_2[0] in model:  
        vec2 = model[sents_2[0]]
      
      else:
        vec2= 0.0
      
      for word2 in sents_2[1:]:
        if word2 in model:
          vec2 = vec2 + model[word2]
      else:
          vec2 = vec2 + 0.0

    similarity = cosine_similarity(vec1 / len(sents_1), vec2 / len(sents_2))
    return similarity  

def calculate_score(weight_graph, scores, i):
    """
    计算句子在图中的分数
    :param weight_graph:
    :param scores:
    :param i:
    :return:
    """
    length = len(weight_graph)
    d = 0.85
    added_score = 0.0
 
    for j in range(length):
        fraction = 0.0
        denominator = 0.0
        # 计算分子
        fraction = weight_graph[j][i] * scores[j]
        # 计算分母
        for k in range(length):
            denominator += weight_graph[j][k]
            #if denominator == 0:
             # denominator = 1
        added_score += fraction / denominator
    # 算出最终的分数
    weighted_score = (1 - d) + d * added_score
    return weighted_score  

def weight_sentences_rank(weight_graph):
    '''
    输入相似度的图（矩阵)
    返回各个句子的分数
    :param weight_graph:
    :return:
    '''
    # 初始分数设置为0.5
    scores = [0.5 for _ in range(len(weight_graph))]
    old_scores = [0.0 for _ in range(len(weight_graph))]
 
    # 开始迭代
    while different(scores, old_scores):
        for i in range(len(weight_graph)):
            old_scores[i] = scores[i]
        for i in range(len(weight_graph)):
            scores[i] = calculate_score(weight_graph, scores, i)
            scores[i] = np.mean(scores[i])
            #print(scores[i])
    return scores

def different(scores, old_scores):
    '''
    判断前后分数有无变化
    :param scores:
    :param old_scores:
    :return:
    '''
    flag = False
    for i in range(len(scores)):
        s = scores[i] - old_scores[i]
        if math.fabs(s) >= 0.0001:
            flag = True
            break
    return flag

def filter_symbols(sents):
    stopwords = create_stopwords() + ['。', ' ', '.']
    _sents = []
    for sentence in sents:
        for word in sentence:
            if word in stopwords:
                sentence.remove(word)
        if sentence:
            _sents.append(sentence)
    return _sents

def filter_symbols(sents):
  #预处理
    stopwords = create_stopwords() + ['。', ' ', '.']
    _sents = []
    for sentence in sents:
        for word in sentence:
            if word in stopwords:
                sentence.remove(word)#2
        if sentence:
            _sents.append(sentence)
    return _sents

def filter_model(sents):
    _sents = []
    for sentence in sents:
      if sentence:
        _sents.append(sentence)
    return _sents

def summarize(text,n):
  tokens = cut_sentences(text)
  #tokens = "2020年の主要16品目の半導体市場規模は、新型コロナウイルス感染症の影響による外出制限に伴うリモートワークの推進などを追い風に、半導体デバイスの需要が増加した結果、前年比14.4％増の26兆678億円が見込まれるという。その後も米中貿易摩擦や日韓貿易摩擦による半導体市場への影響が見受けられるものの、リモートワークの一般化、AIの普及などに伴い、半導体の活用が進むとみられ、2025年にはその市場規模は2019年比88.9％増の43兆470億円に到達するものと同社は予測している。品目別にみると、DRAMの市場規模がもっとも大きく、次いでNAND、モバイル機器用APと続く。DRAMやNANDは、データセンターの投資が好調であることから継続的に伸長していくとみられる。スマートフォン(スマホ)やスマートグラスなどに搭載されるモバイル機器用APは、5G通信のサービス開始に伴い5G対応スマートフォンの需要が高まっていることや、スマートウォッチやスマートグラスなどへ用途が広がっていることから、2021年以降市場は堅調に拡大していくと予想されるという。もっとも大きな市場として期待されるDRAMの2020年市場について同社は、前年比19.4％増の8兆円と見込んでいる。CPUの開発遅延などにより投資が緩やかであったデータセンターにおいて、リモートワークの普及によるデータ通信料の増加や代替CPUの登場によって投資が活発化しており、今後も投資が継続していくことが期待されるため、2025年には2019年比94.0％増の13兆円に達すると予測している。メモリ分野のもう1つの大市場であるNANDの2020年市場規模は前年比34.9％増の6兆8000億円が見込まれると富士キメラでは見込んでいる。また、今後もデータセンターを中心にストレージに対する投資が進むことから、需要は堅調に推移し、2025年には2019年比で3.1倍となる15兆5000億円に達すると予測している。半導体業界が期待してきた自動車分野の中核の1つ、自動車用SoCの2020年市場は、新型コロナの影響で自動車販売が不振に陥っていることから前年比12.2％減の2030億円となると見込まれているが、今後は自動運転レベルの向上に伴い、1台当たりの搭載個数は増加していくこととなり、需要の増加が期待されることから、2025年には2019年比2.0倍の4659億円に達すると予測している。スマートフォンへの搭載数拡大のほか、自動車での活用も進むイメージセンサ市場について同社は、2020年が前年比3.0％増の1兆9680億円と見込んでいる。現在、スマートフォンへの搭載数が増加しているが、今後もそのトレンドは継続するほか、自動車でも搭載数が増加していくと予測されるため、市場の拡大が続くことが期待され、2025年には2019年比38.5％増の2兆6460億円に達すると予測している。そしてセンサ系としては、ToFセンサについて同社は、5Gや画像認識技術の進化に伴い、ARの普及が進むことが期待されており、2020年の市場規模は前年比6.4％増の1055億円と見込んでいる。また、スマホへの搭載は当然ながら、中長期的にはスマートグラスへの搭載も進むことが期待されているため、2025年には2019年比3.2倍増の3143億円と予測している。"
  sentences = []
  sents = []
  mecabTagger = MeCab.Tagger("-Owakati")
  for sent in tokens:
    sentences.append(sent)
    #print("sent"+sent)
    #print(mecabTagger.parse(sent))
    sents.append(mecabTagger.parse(sent))
    #print("sent1"+sents)
  sents = filter_model(sents)
  graph = create_graph(sents)

  scores = weight_sentences_rank(graph)
  sent_selected = nlargest(n, zip(scores, count()))
  sent_index = []
  for i in range(n):
      sent_index.append(sent_selected[i][1])#bug
  return [sentences[i] for i in sent_index] #bug cosine_similarity test

In [ ]:
import subprocess

m=MeCab.Tagger("-Owakati")
node = m.parse("富士通と東京品川病院は9月2日、新型コロナウイルス肺炎の診断に有効とされる胸部CT(Computed Tomography:コンピューター断層撮影)検査による画像診断の支援を行うAI技術の共同研究開発を同日から実施すると明らかにした。新型コロナウイルス感染の疑いが強い患者の治療を行う際、PCR検査結果だけでなく、血液検査や胸部CT検査など、そのほかの検査結果を踏まえて総合的に診断および治療方針の決定を行う。PCR検査が陰性でも、そのほかの検査で新型コロナウイルス肺炎と診断される場合もあり、胸部CT検査による画像診断は重要な位置づけとなっているという。肺疾患の診断では、医師が患者の胸部CT画像に映った病変部の陰影の特徴に基づき診断しているが、異常な陰影の判別だけでなく、肺全体にわたる陰影の立体的な分布状況を把握するため、患者一人あたり数百枚にもおよぶ胸部CT画像を目視で確認している。特に、新型コロナウイルス感染症拡大に伴い、医師の負担を軽減し、スピーディーな診断を支援する技術が求められているほか、問診で新型コロナウイルス感染の可能性が低く、PCR検査が実施されない場合、その患者の胸部CT画像から新型コロナウイルス肺炎を見つけ出すことも重要だという。これらの診断支援の必要性から両者は共同で、新型コロナウイルス肺炎の診断に有効な手法とされる胸部CT検査に対して、AIを活用した医師の画像診断支援技術の開発を開始。具体的には、東京品川病院が有する過去の新型コロナウイルス肺炎の胸部CT画像データから肺の異常陰影パターンを検出して、それらのデータをAIに学習させることで新型コロナウイルス肺炎の可能性を示すAI技術を開発し、その技術の有効性を両者で検証する。医師が新型コロナウイルス肺炎を診断する際、肺に見られる異常陰影のパターンと肺全体の異常陰影の広がり方が診断のための重要な情報となっており、異常陰影のパターンの検出は富士通研究所が開発したAIを活用して行い、CT画像上で肺を右肺末梢、右肺中枢、左肺中枢、左肺末梢の4つの領域に分割し、各領域の上下方向の陰影分布をヒストグラム化(データの分布状況を視覚的に認識可能な統計グラフの1つ)する。これにより、三次元的な陰影の広がりを数値化し、検出された異常陰影パターンと陰影分布を用いて新型コロナウイルス肺炎を判別するAIを新たに開発。AIで新型コロナウイルス肺炎の可能性を示すことで、医師が")
node = mt.parse("昨日、お笑い番組を見た。")
#print(node)

for word in node.split():
  #print(word)
  print(m[word])
  print(m[word].shape)

TypeError: ignored

In [ ]:
#model = pickle.load('lstm_datamodel.pkl')

In [ ]:
with open("/content/news.txt", "r", encoding='utf-8') as myfile:
        text = myfile.read().replace('\n', '')
        summarys=summarize("2020年の主要16品目の半導体市場規模は、新型コロナウイルス感染症の影響による外出制限に伴うリモートワークの推進などを追い風に、半導体デバイスの需要が増加した結果、前年比14.4％増の26兆678億円が見込まれるという。その後も米中貿易摩擦や日韓貿易摩擦による半導体市場への影響が見受けられるものの、リモートワークの一般化、AIの普及などに伴い、半導体の活用が進むとみられ、2025年にはその市場規模は2019年比88.9％増の43兆470億円に到達するものと同社は予測している。品目別にみると、DRAMの市場規模がもっとも大きく、次いでNAND、モバイル機器用APと続く。DRAMやNANDは、データセンターの投資が好調であることから継続的に伸長していくとみられる。スマートフォン(スマホ)やスマートグラスなどに搭載されるモバイル機器用APは、5G通信のサービス開始に伴い5G対応スマートフォンの需要が高まっていることや、スマートウォッチやスマートグラスなどへ用途が広がっていることから、2021年以降市場は堅調に拡大していくと予想されるという。もっとも大きな市場として期待されるDRAMの2020年市場について同社は、前年比19.4％増の8兆円と見込んでいる。CPUの開発遅延などにより投資が緩やかであったデータセンターにおいて、リモートワークの普及によるデータ通信料の増加や代替CPUの登場によって投資が活発化しており、今後も投資が継続していくことが期待されるため、2025年には2019年比94.0％増の13兆円に達すると予測している。メモリ分野のもう1つの大市場であるNANDの2020年市場規模は前年比34.9％増の6兆8000億円が見込まれると富士キメラでは見込んでいる。また、今後もデータセンターを中心にストレージに対する投資が進むことから、需要は堅調に推移し、2025年には2019年比で3.1倍となる15兆5000億円に達すると予測している。半導体業界が期待してきた自動車分野の中核の1つ、自動車用SoCの2020年市場は、新型コロナの影響で自動車販売が不振に陥っていることから前年比12.2％減の2030億円となると見込まれているが、今後は自動運転レベルの向上に伴い、1台当たりの搭載個数は増加していくこととなり、需要の増加が期待されることから、2025年には2019年比2.0倍の4659億円に達すると予測している。スマートフォンへの搭載数拡大のほか、自動車での活用も進むイメージセンサ市場について同社は、2020年が前年比3.0％増の1兆9680億円と見込んでいる。現在、スマートフォンへの搭載数が増加しているが、今後もそのトレンドは継続するほか、自動車でも搭載数が増加していくと予測されるため、市場の拡大が続くことが期待され、2025年には2019年比38.5％増の2兆6460億円に達すると予測している。そしてセンサ系としては、ToFセンサについて同社は、5Gや画像認識技術の進化に伴い、ARの普及が進むことが期待されており、2020年の市場規模は前年比6.4％増の1055億円と見込んでいる。また、スマホへの搭載は当然ながら、中長期的にはスマートグラスへの搭載も進むことが期待されているため、2025年には2019年比3.2倍増の3143億円と予測している。",1)
        print(summarys)
        for each in summarys:
            print (each+"\n"+"*"*10)

ここまでは自分でTEXTRANKアルゴリズムの実現です。計算スピードが遅くためラブライブを用いて要約をやってみます


In [ ]:
model_path = '/content/word2vec.gensim.model'
m = Word2Vec.load(model_path)
#wv = KeyedVectors.load_word2vec_format(model_path, binary=True,encoding="utf-8")
word_embeddings = {}
print(m["新型コロナウイルス"].shape)

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:254: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


(50,)


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  """


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
#sentence類似度計算
from gensim.models.word2vec import Word2Vec
from gensim.models import KeyedVectors

model_path = '/content/word2vec.gensim.model'
m = Word2Vec.load(model_path)
#wv = KeyedVectors.load_word2vec_format(model_path, binary=True,encoding="utf-8")
word_embeddings = {}

mt = MeCab.Tagger("-Owakati")
#print(m["富士通"])
#node = mt.parse("昨日、お笑い番組を見た。")


def get_vector(text):
  sum_vec =np.zeros(50)
  word_count = 0
  node = mt.parse(text)
  #print(node)
  for word in node.split():
    print(word)
    sum_vec +=m[word]
    word_count += 1
    #word = word.next
  return sum_vec/word_count

"""
  while node:
    print(node+"/n")
    #fields = node.feature.split(",")
    sum_vec +=m[node]
    word_count += 1
    node = node.next
  return sum_vec/word_count
"""

#print(get_vector("富士通と東京品川病院は9月2日、新型コロナウイルス肺炎の診断に有効とされる胸部CT(Computed Tomography:コンピューター断層撮影)検査による画像診断の支援を行うAI技術の共同研究開発を同日から実施すると明らかにした。新型コロナウイルス感染の疑いが強い患者の治療を行う際、PCR検査結果だけでなく、血液検査や胸部CT検査など、そのほかの検査結果を踏まえて総合的に診断および治療方針の決定を行う。PCR検査が陰性でも、そのほかの検査で新型コロナウイルス肺炎と診断される場合もあり、胸部CT検査による画像診断は重要な位置づけとなっているという。肺疾患の診断では、医師が患者の胸部CT画像に映った病変部の陰影の特徴に基づき診断しているが、異常な陰影の判別だけでなく、肺全体にわたる陰影の立体的な分布状況を把握するため、患者一人あたり数百枚にもおよぶ胸部CT画像を目視で確認している。特に、新型コロナウイルス感染症拡大に伴い、医師の負担を軽減し、スピーディーな診断を支援する技術が求められているほか、問診で新型コロナウイルス感染の可能性が低く、PCR検査が実施されない場合、その患者の胸部CT画像から新型コロナウイルス肺炎を見つけ出すことも重要だという。これらの診断支援の必要性から両者は共同で、新型コロナウイルス肺炎の診断に有効な手法とされる胸部CT検査に対して、AIを活用した医師の画像診断支援技術の開発を開始。具体的には、東京品川病院が有する過去の新型コロナウイルス肺炎の胸部CT画像データから肺の異常陰影パターンを検出して、それらのデータをAIに学習させることで新型コロナウイルス肺炎の可能性を示すAI技術を開発し、その技術の有効性を両者で検証する。医師が新型コロナウイルス肺炎を診断する際、肺に見られる異常陰影のパターンと肺全体の異常陰影の広がり方が診断のための重要な情報となっており、異常陰影のパターンの検出は富士通研究所が開発したAIを活用して行い、CT画像上で肺を右肺末梢、右肺中枢、左肺中枢、左肺末梢の4つの領域に分割し、各領域の上下方向の陰影分布をヒストグラム化(データの分布状況を視覚的に認識可能な統計グラフの1つ)する。これにより、三次元的な陰影の広がりを数値化し、検出された異常陰影パターンと陰影分布を用いて新型コロナウイルス肺炎を判別するAIを新たに開発。AIで新型コロナウイルス肺炎の可能性を示すことで、医師が"))
def cos_sim(v1, v2):
    return np.dot(v1, v2) / (np.linalg.norm(v1) * np.linalg.norm(v2))


if __name__ == "__main__":
    v1 = get_vector('昨日、お笑い番組を見た。')
    v2 = get_vector('昨夜、テレビで漫才をやっていた。')
    v3 = get_vector('昨日、公園に行った。')

    print(cos_sim(v1, v2))
    print(cos_sim(v1, v3))

本作業

In [ ]:
a = open("/content/test.txt","r")
sentence_word_list = []
for sentence in a.readlines():
    line_seg = sentence
    sentence_word_list.append(line_seg)
print("一共有",len(sentence_word_list),'个句子。\n')
print(sentence_word_list)

一共有 13 个句子。

['富士通と東京品川病院は9月2日、新型コロナウイルス肺炎の診断に有効とされる胸部CT(sコンピューター断層撮影)検査による画像診断の支援を行うAI技術の共同研究開発を同日から実施すると明らかにした。\n', '新型コロナウイルス感染の疑いが強い患者の治療を行う際、PCR検査結果だけでなく、血液検査や胸部CT検査など、そのほかの検査結果を踏まえて総合的に診断および治療方針の決定を行う。\n', 'PCR検査が陰性でも、そのほかの検査で新型コロナウイルス肺炎と診断される場合もあり、胸部CT検査による画像診断は重要な位置づけとなっているという。\n', '肺疾患の診断では、医師が患者の胸部CT画像に映った病変部の陰影の特徴に基づき診断しているが、異常な陰影の判別だけでなく、肺全体にわたる陰影の立体的な分布状況を把握するため、患者一人あたり百枚にもおよぶ胸部CT画像を目視で確認している。\n', '特に、新型コロナウイルス感染症拡大に伴い、医師の負担を軽減し、スピーディーな診断を支援する技術が求められているほか、問診で新型コロナウイルス感染の可能性が低く、PCR検査が実施されない場合、その患者の胸部CT画像から新型コロナウイルス肺炎を見つけ出すことも重要だという。\n', 'これらの診断支援の必要性から両者は共同で、新型コロナウイルス肺炎の診断に有効な手法とされる胸部CT検査に対して、AIを活用した医師の画像診断支援技術の開発を開始。\n', '具体的には、東京品川病院が有する過去の新型コロナウイルス肺炎の胸部CT画像データから肺の異常陰影パターンを検出して、それらのデータをAIに学習させることで新型コロナウイルス肺炎の可能性を示すAI技術を開発し、その技術の有効性を両者で検証する。\n', '医師が新型コロナウイルス肺炎を診断する際、肺に見られる異常陰影のパターンと肺全体の異常陰影の広がり方が診断のための重要な情報となっており、異常陰影のパターンの検出は富士通研究所が開発したAIを活用して行い、CT画像上で肺を肺末梢、肺中枢、肺中枢、肺末梢の4つの領域に分割し、各領域の上下方向の陰影分布をヒストグラム化(データの分布状況を視覚的に認識可能な統計グラフの1つ)する。\n', 'これにより、三次元的な陰影の広がりを数値化し、検出された異常陰影パター

In [ ]:
#句子向量
from gensim.models.word2vec import Word2Vec
from gensim.models import KeyedVectors
import MeCab

model_path = '/content/word2vec.gensim.model'
m = Word2Vec.load(model_path)
#wv = KeyedVectors.load_word2vec_format(model_path, binary=True,encoding="utf-8")
word_embeddings = {}

mt = MeCab.Tagger("-Owakati")
#print(m["富士通"])
#node = mt.parse("昨日、お笑い番組を見た。")


def get_vector(text):
  sum_vec =np.zeros(50)
  word_count = 0
  node = mt.parse(text)
  #print(node)
  for word in node.split():
    #print(word)
    sum_vec +=m[word]
    word_count += 1
    #word = word.next
    #print(sum_vec)

In [ ]:
問題点：sentence毎にvector化しているので、ドキュメントはダメ

In [ ]:
#句子矩阵

#print(len(a.readlines()))
sentence_vectors = []
for i in a.readlines():
  print(get_vector(i))
  sentence_vectors.append(get_vector(i))
#print(sentence_vectors)
#sentence_vectors -->list

In [ ]:
#句子相似矩阵
import numpy as np

sim_mat = np.zeros([13,13])

from sklearn.metrics.pairwise import cosine_similarity

for i in range(13):
  for j in range(13):
    if i != j:
      sim_mat[i][j] = cosine_similarity(sentence_vectors[i].reshape(1,50), sentence_vectors[j].reshape(1,50))[0,0]
print("句子相似度矩阵的形状为：",sim_mat.shape)

IndexError: ignored

In [ ]:
"""第五步：迭代得到句子的textrank值，排序并取出摘要"""
import networkx as nx

# 利用句子相似度矩阵构建图结构，句子为节点，句子相似度为转移概率
nx_graph = nx.from_numpy_array(sim_mat)

# 得到所有句子的textrank值
scores = nx.pagerank(nx_graph)

# 根据textrank值对未处理的句子进行排序
ranked_sentences = sorted(((scores[i],s) for i,s in enumerate(sentence_word_list)), reverse=True)

# 取出得分最高的前10个句子作为摘要
sn = 3
for i in range(sn):
    print("第"+str(i+1)+"条摘要：\n\n",ranked_sentences[i][1],'\n')

第1条摘要：

 具体的には、東京品川病院が有する過去の新型コロナウイルス肺炎の胸部CT画像データから肺の異常陰影パターンを検出して、それらのデータをAIに学習させることで新型コロナウイルス肺炎の可能性を示すAI技術を開発し、その技術の有効性を両者で検証する。
 

第2条摘要：

 AIで新型コロナウイルス肺炎の可能性を示すことで、医師が胸部CT画像から肺炎の診断をする際、陰影の立体的な広がりを百枚の胸部CT画像から目視で確認していた診断時間を短縮し、専門医に加え、新型コロナウイルス肺炎の診断の効率化を図る。
 

第3条摘要：

 これらの診断支援の必要性から両者は共同で、新型コロナウイルス肺炎の診断に有効な手法とされる胸部CT検査に対して、AIを活用した医師の画像診断支援技術の開発を開始。
 



In [ ]:
#分割句子 未实现
a = open("/content/news.txt","r")
#print(a.readline(0))
b = open("/content/renews.txt","w")
for line in a.readlines(0):
  print(line.split("。"))
  b.write(str(line.split("。")))
  b.write("\n")
b.close()

['富士通と東京品川病院は9月2日、新型コロナウイルス肺炎の診断に有効とされる胸部CT(Computed Tomography:コンピューター断層撮影)検査による画像診断の支援を行うAI技術の共同研究開発を同日から実施すると明らかにした', '新型コロナウイルス感染の疑いが強い患者の治療を行う際、PCR検査結果だけでなく、血液検査や胸部CT検査など、そのほかの検査結果を踏まえて総合的に診断および治療方針の決定を行う', 'PCR検査が陰性でも、そのほかの検査で新型コロナウイルス肺炎と診断される場合もあり、胸部CT検査による画像診断は重要な位置づけとなっているという', '肺疾患の診断では、医師が患者の胸部CT画像に映った病変部の陰影の特徴に基づき診断しているが、異常な陰影の判別だけでなく、肺全体にわたる陰影の立体的な分布状況を把握するため、患者一人あたり数百枚にもおよぶ胸部CT画像を目視で確認している', '特に、新型コロナウイルス感染症拡大に伴い、医師の負担を軽減し、スピーディーな診断を支援する技術が求められているほか、問診で新型コロナウイルス感染の可能性が低く、PCR検査が実施されない場合、その患者の胸部CT画像から新型コロナウイルス肺炎を見つけ出すことも重要だという', 'これらの診断支援の必要性から両者は共同で、新型コロナウイルス肺炎の診断に有効な手法とされる胸部CT検査に対して、AIを活用した医師の画像診断支援技術の開発を開始', '具体的には、東京品川病院が有する過去の新型コロナウイルス肺炎の胸部CT画像データから肺の異常陰影パターンを検出して、それらのデータをAIに学習させることで新型コロナウイルス肺炎の可能性を示すAI技術を開発し、その技術の有効性を両者で検証する', '医師が新型コロナウイルス肺炎を診断する際、肺に見られる異常陰影のパターンと肺全体の異常陰影の広がり方が診断のための重要な情報となっており、異常陰影のパターンの検出は富士通研究所が開発したAIを活用して行い、CT画像上で肺を右肺末梢、右肺中枢、左肺中枢、左肺末梢の4つの領域に分割し、各領域の上下方向の陰影分布をヒストグラム化(データの分布状況を視覚的に認識可能な統計グラフの1つ)する', 'これにより、三次元的な陰影の広がりを数値化し、検出された異常陰影パターンと陰影分布を用いて新型コ

In [ ]:
##to-do 把上面的功能进行汇总
from gensim.models.word2vec import Word2Vec
from gensim.models import KeyedVectors

model_path = '/content/word2vec.gensim.model'
m = Word2Vec.load(model_path)
#wv = KeyedVectors.load_word2vec_format(model_path, binary=True,encoding="utf-8")
word_embeddings = {}

mt = MeCab.Tagger("-Owakati")

a = open("/content/test.txt","r")
for i in a.readlines():
  print(get_vector(i))

#sim_mat = np.zeros([len(sentences_list), len(sentences_list)])
#類似度の計算はこち使うかな？
from sklearn.metrics.pairwise import cosine_similarity

sim_mat = np.zeros([len(sentences_list), len(sentences_list)])
a = open("/content/test.txt","r")
for i in range(len(13)):
  for j in range(len(13)):
    sim_mat[i][j] = cosine_similarity(get_vector[i].reshape(1,50), get_vector[j].reshape(1,50))[0,0] 


import networkx as nx

nx_graph = nx.from_numpy_array(sim_mat)
scores = nx.pagerank(nx_graph)

ranked_sentences = sorted(((scores[i],s) for i,s in enumerate(sentences)), reverse=True)

# Specify number of sentences to form the summary
sn = 10

# Generate summary
for i in range(sn):
  print(ranked_sentences[i][1])


/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:254: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


NameError: ignored

In [ ]:
a = np.array([[0.25,0.3,0.363,0.512],[0.12,0.25,0.33,0.15],[1.2,2.3,0.36,0.44]])
a
print(a.mean(axis=1))

[0.35625 0.2125  1.075  ]
